# **Simplex algorithm**

Implementation of the simplex algorithm

In [1]:
import numpy as np
from scipy import linalg as la

In [2]:
# Implementation of simplex algorithm
# Problem considered in the standard form
# minimize c'x subject to Ax <= b
def simplex_algorithm( top, c, A, b ) :
  [ m, n ] = A.shape

  # Reduced costs
  if top == 'max' :
    cr = -c
  elif top == 'min' :
    cr = c
  
  cr = np.insert( cr, 0, 0, axis = 0 )

  # Preparing the simplex table
  X = np.insert( A, 0, b, axis = 1 )
  X = np.insert( X, 0, cr, axis = 0 )

  # Slack variables
  S = np.eye( m )    
  S = np.insert( S, 0, np.zeros( m ), axis = 0 )
  X = np.insert( X, np.repeat( n + 1, m ), S, axis = 1 )
  [ p, q ] = X.shape
  
  # Simplex tables
  ST = []
  ST += [ np.copy( X ) ]
  
  # Feasible base
  Bch = np.arange( q - m, q, 1 )
  B = np.arange( q - m, q, 1 )
  B.shape = [1,m]
  
  cj = -np.inf
  while cj < 0 :

    # Selection pivot column and the smaller reduced cost
    cj = X[0,1]
    j = 1
    for l in range( 2, q ) :
      if X[0,l] < cj :
        j = l
        cj = X[0,l]

    # Proceed if the smaller reduced cost is negative
    if cj < 0 :
      theta = np.inf
      i = -1

      # Selection of pivot row
      for k in range( 1, p ) :
        if X[k,j] > 0 :
          a = X[k,0] / X[k,j] 
          if a >= 0 and a < theta:
            theta = a
            i = k

      # Updating evolution of feasible base
      Bch[ i - 1 ] = j - 1
      B = np.insert( B, B.shape[0], Bch, axis = 0 )
      
      r = 1.0 / X[i,j]
      X[i,:] = r * X[i,:]

      # Pivoting
      for k in range( 0, p ) :
        if k != i :
          r = X[k,j]
          X[k,:] = X[k,:] - r * X[i,:]
      
      ST += [ np.copy( X ) ]

  x = { i : 0 for i in range( 0, q - 1 ) }
  for i in range( 0, m ) :
    x[ Bch[i] ] = X[i+1,0] 
  
  if top == 'min' :
    z = -X[0,0]
  elif top == 'max' :
    z = X[0,0]
  
  return [ x, z, ST, B ]

## **Ejemplo 1**

In [3]:
n = 3
A = np.eye( n, dtype = np.double )
b = np.ones( n, dtype = np.double )
c = np.ones( n, dtype = np.double )

ST = simplex_algorithm( 'max', c, A, b )

In [4]:
print( 'Optimal point:\n', ST[0] )

Optimal point:
 {0: 1.0, 1: 1.0, 2: 1.0, 3: 0, 4: 0, 5: 0}


In [5]:
print( 'Optimal value:\n', ST[1] ) 

Optimal value:
 3.0


In [6]:
print( 'Evolution of simplex tables:\n' )
for i in range( 0, len( ST[2] ) ) :
  print( ST[2][i], '\n' )

Evolution of simplex tables:

[[ 0. -1. -1. -1.  0.  0.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.]
 [ 1.  0.  1.  0.  0.  1.  0.]
 [ 1.  0.  0.  1.  0.  0.  1.]] 

[[ 1.  0. -1. -1.  1.  0.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.]
 [ 1.  0.  1.  0.  0.  1.  0.]
 [ 1.  0.  0.  1.  0.  0.  1.]] 

[[ 2.  0.  0. -1.  1.  1.  0.]
 [ 1.  1.  0.  0.  1.  0.  0.]
 [ 1.  0.  1.  0.  0.  1.  0.]
 [ 1.  0.  0.  1.  0.  0.  1.]] 

[[3. 0. 0. 0. 1. 1. 1.]
 [1. 1. 0. 0. 1. 0. 0.]
 [1. 0. 1. 0. 0. 1. 0.]
 [1. 0. 0. 1. 0. 0. 1.]] 



In [7]:
print( 'Evolution of the feasible base:\n', ST[3] )

Evolution of the feasible base:
 [[4 5 6]
 [0 5 6]
 [0 1 6]
 [0 1 2]]


## **Ejemplo 2**

In [8]:
A = np.array( [ [1, 2,2], [2,1,2], [2,2,1] ], dtype = np.double )
b = np.array( [ 20, 20, 20 ], dtype = np.double )
c = np.array( [ -10, -12, -12 ], dtype = np.double )

ST = simplex_algorithm( 'min', c, A, b )

In [9]:
print( 'Optimal point:\n', ST[0] )

Optimal point:
 {0: 4.0, 1: 4.0, 2: 4.0, 3: 0, 4: 0, 5: 0}


In [10]:
print( 'Optimal value:\n', ST[1] ) 

Optimal value:
 -136.0


In [11]:
print( 'Evolution of simplex tables:\n' )
for i in range( 0, len( ST[2] ) ) :
  print( ST[2][i], '\n' )

Evolution of simplex tables:

[[  0. -10. -12. -12.   0.   0.   0.]
 [ 20.   1.   2.   2.   1.   0.   0.]
 [ 20.   2.   1.   2.   0.   1.   0.]
 [ 20.   2.   2.   1.   0.   0.   1.]] 

[[120.   -4.    0.    0.    6.    0.    0. ]
 [ 10.    0.5   1.    1.    0.5   0.    0. ]
 [ 10.    1.5   0.    1.   -0.5   1.    0. ]
 [  0.    1.    0.   -1.   -1.    0.    1. ]] 

[[120.    0.    0.   -4.    2.    0.    4. ]
 [ 10.    0.    1.    1.5   1.    0.   -0.5]
 [ 10.    0.    0.    2.5   1.    1.   -1.5]
 [  0.    1.    0.   -1.   -1.    0.    1. ]] 

[[136.    0.    0.    0.    3.6   1.6   1.6]
 [  4.    0.    1.    0.    0.4  -0.6   0.4]
 [  4.    0.    0.    1.    0.4   0.4  -0.6]
 [  4.    1.    0.    0.   -0.6   0.4   0.4]] 



In [12]:
print( 'Evolution of the feasible base:\n', ST[3] )

Evolution of the feasible base:
 [[4 5 6]
 [1 5 6]
 [1 5 0]
 [1 2 0]]
